In [1]:
proj_path='/content/drive/MyDrive/S4 - 16824 VLR/generative-modeling/gan'

### Git Stuff

### Colab stuff

In [ ]:
# one-time data set-up
# #!pip show "clean-fid"
# !cd "$proj_path/$hw_dir" && source "setup_data.sh"
# !ls "/usr/local/lib/python3.7/dist-packages/cleanfid"
# !mkdir "/usr/local/lib/python3.7/dist-packages/cleanfid/stats"
# !cd "$proj_path/$hw_dir/$tasks" && cp cub_clean_custom_na.npz "/usr/local/lib/python3.7/dist-packages/cleanfid/stats/cub_clean_custom_na.npz"

## Notebook

In [ ]:
import sys, pathlib
sys.path.append('./gan')
sys.path.append('./vae')
%load_ext autoreload
%autoreload 2

### GAN

In [ ]:
from gan.networks import *
from gan.train import *
from gan.q1_3 import compute_discriminator_loss, compute_generator_loss
from networks import Discriminator, Generator, UpSampleConv2D, DownSampleConv2D

In [4]:
x = torch.randn(1,3,32,32).cuda()
gen = Generator().cuda()
disc = Discriminator().cuda()
prefix = "data_gan/"

In [ ]:
#prefix = "data_wgan_gp/"
os.makedirs(prefix, exist_ok=True)
# TODO 1.3.2: Run this line of code.
train_model(
    gen,
    disc,
    num_iterations=int(3e4),
    batch_size=128,
    prefix=prefix,
    gen_loss_fn=compute_generator_loss,
    disc_loss_fn=compute_discriminator_loss,
    log_period=1000,
    wandb_logging=True
)

In [ ]:
print('done')

### VAE

In [4]:
import sys, pathlib, os
sys.path.append(os.getcwd()+'/vae')
%load_ext autoreload
%autoreload 2

In [15]:
from vae.model import AEModel, VAEEncoder
from vae.train import vae_loss
#from vae.utils import compute_discriminator_loss, compute_generator_loss
#from networks import Discriminator, Generator

In [20]:
input_shape = (3, 32, 32)
x = torch.randn((2,)+input_shape).cuda()
variational = True
model = AEModel(variational=variational, latent_size=1024, input_shape = (3, 32, 32)).cuda()
if variational:
    mu, logvar = model.encoder(x)
    print(mu.size(), logvar.size())
    print(vae_loss(model, x))
else:
    mu = model.encoder(x)
    print(ae_loss(model, x))
print(model.decoder(torch.randn(2, 1024).cuda()).size())
print(get_val_metrics(model, 'vae', val_loader))

torch.Size([2, 1024]) torch.Size([2, 1024])
(tensor(4415.4678, device='cuda:0', grad_fn=<AddBackward0>), OrderedDict([('srecon_loss', tensor(4415.1357, device='cuda:0', grad_fn=<MulBackward0>)), ('kl_loss', tensor(0.3318, device='cuda:0', grad_fn=<MeanBackward0>))]))
torch.Size([2, 3, 32, 32])


In [ ]:
import argparse 
def parse_a2c_arguments():
    # Command-line flags are defined here.
    parser = argparse.ArgumentParser()
    parser.add_argument('--latent_size', dest='latent_size', type=int,
                        default=1024, help="Size of latent space")   # 'LunarLander-v2'
    parser.add_argument('--num_epochs', dest='num_epochs', type=int,
                        default=20, help="Size of latent space")   # 'LunarLander-v2'
    parser.add_argument('--loss_mode', dest='loss_mode', type=str,
                        default='ae', help="Size of latent space")   # 'LunarLander-v2'
    parser.add_argument('--log_dir', dest='log_dir', type=str,
                        default='ae_latent1024', help="directory")
    # ['ae_latent1024','vae_latent1024', 'vae_latent1024_beta_constant0.8','vae_latent1024_beta_linear1']
    parser.add_argument('--beta_mode', dest='beta_mode', type=str,
                        default='constant', help="directorye")   
    # ['constant', 'linear']
    parser.add_argument('--target_beta_val', dest='target_beta_val', type=float,
                        default=0.8, help="final beta")   # 
    # [0.8. 1]
    return parser.parse_known_args()[0]  #parser.parse_args()
args = parse_a2c_arguments()
print(args)

In [ ]:
main(*args.__dict__)